# Data Extraction from API

In [ ]:
import pandas as pd

## BKK Policy Tracking Webscrapping

In [ ]:
TOP_30_POLICIES_URL = 'https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findex'

# Read HTML tables into a list of DataFrame objects.
all_tables = pd.read_html(TOP_30_POLICIES_URL)
# Check number of table(s)
len(all_tables)

KeyboardInterrupt: 

In [ ]:
df_rushing_policy = all_tables[0]
df_rushing_policy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   เป้าหมายตามนโยบาย ผว.กทม.  30 non-null     object 
 1   หน่วยนับ                   30 non-null     object 
 2   เป้าหมายรวม                30 non-null     float64
 3   เป้าหมายจากสำนัก           30 non-null     float64
 4   เป้าหมายจาก 50 เขต         30 non-null     float64
 5   ผลการดำเนินงาน*            30 non-null     float64
 6   ความคืบหน้า* ของ KPI (%)   31 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.8+ KB


In [63]:
pd.set_option('display.float_format', str)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_rushing_policy.columns

Index(['เป้าหมายตามนโยบาย ผว.กทม.', 'หน่วยนับ', 'เป้าหมายรวม',
       'เป้าหมายจากสำนัก', 'เป้าหมายจาก 50 เขต', 'ผลการดำเนินงาน*',
       'ความคืบหน้า* ของ KPI (%)'],
      dtype='object')

In [ ]:
df_rushing_policy.dropna(axis = 0, inplace = True)

new_name = {'เป้าหมายตามนโยบาย ผว.กทม.': 'Goal', 'หน่วยนับ':'Unit', 'เป้าหมายรวม': 'Total Goals',
       'เป้าหมายจากสำนัก':'Goal_Departments', 'เป้าหมายจาก 50 เขต':'Goal_50_Districts', 'ผลการดำเนินงาน*':'Total_Progress_Unit',
       'ความคืบหน้า* ของ KPI_in_Percent':'Progress_in_Percent'}

df_rushing_policy.rename(columns = new_name, inplace = True)

# new data frame with split value columns
new = df_rushing_policy["Goal"].str.split(":", n=1, expand=True)

# making separate first name column from new data frame
df_rushing_policy["Goal"] = new[0]

# making separate last name column from new data frame
df_rushing_policy["Related OKR"] = new[1]

# Dropping old Name columns
# df_rushing_policy.drop(columns=["Goal"], inplace=True)

df_rushing_policy['Related OKR'] = df_rushing_policy['Related OKR'].str.replace('OKR ', '')
df_rushing_policy['Related OKR'] = df_rushing_policy['Related OKR'].str.replace('(** ค่าเฉลี่ย **)', '(Mean Value)')

new_2 = df_rushing_policy["Goal"].str.split(" ", n=1, expand=True)
df_rushing_policy["ID_Result"] = new_2[0]
df_rushing_policy["Goal"] = new_2[1]

df_rushing_policy = df_rushing_policy[['ID_Result', 'Related OKR', 'Goal', 'Unit', 'Total Goals', 'Goal_Departments',
       'Goal_50_Districts', 'Total_Progress_Unit', 'Progress_in_Percent']]

df_rushing_policy['ID_Result'] = df_rushing_policy['ID_Result'].str.strip('.')

KeyError: 1

In [ ]:
df_rushing_policy.head(10)

,No. (Goal),Related OKR,Goal,Unit,Total Goals,Goal (Departments),Goal (50 Districts),Total Progress (Unit),Progress (%)
0,1,4.1.10,พัฒนาถนนสวย (กม.),กม.,133.35,0.0,133.35,126.56,94.91
1,2,4.1.01,ปลูกต้นไม้รวมทุกประเภท (ต้น),ต้น,200000.00,50000.0,150000.00,359707.00,179.85
2,3,4.1.05,เพิ่มสวน 15 นาที (แห่ง),แห่ง,156.00,6.0,150.00,127.00,81.41
3,4.1,1.1.05,ปรับปรุงทางเท้า (กม.),กม.,324.11,300.0,24.11,110.67,34.15
4,4.2,1.3.07,ทางจักรยาน (กม.),กม.,518.00,18.0,500.00,228.62,44.14
5,4.3,None,ปรับปรุงซอย (กม.),กม.,136.00,0.0,136.00,75.82,55.76
6,5,2.1.20,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),ดวง,32078.00,13100.0,18978.00,40588.00,126.53
7,6,1.1.03,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),ดวง,8056.00,2114.0,5942.00,4800.00,59.58
8,7,1.1.01,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน...,แห่ง,109.00,0.0,109.00,115.00,105.50
9,8,7.2.05,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รอง...,แห่ง,28.00,0.0,28.00,26.00,92.85


In [ ]:
from datetime import datetime
import pytz

today = datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%d-%m-%Y")
print(today)

10-08-2024


In [ ]:
top30_output_path = "gs://bkk-policy-data/Top_30_policy/top-policy-" + str(today) + ".parquet"
print(top30_output_path)

gs://bkk-policy-data/Top_30_policy/top-policy-07-08-2024.parquet


In [ ]:
type(str(today))

str

In [ ]:
df_rushing_policy.to_csv('df_rushing_policies.csv')

In [ ]:
df_rushing_policy.to_parquet('df_rushing_policies.parquet')

Date Column

In [ ]:
from datetime import date

In [ ]:
today = date.today().strftime("%d-%m-%Y")

In [ ]:
today = date.today().strftime("%d-%m-%Y")

### All code of top30_policy

In [2]:
import pandas as pd
from datetime import datetime
import pytz

In [31]:
TOP_30_POLICIES_URL = 'https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findex'

# Read HTML tables into a list of DataFrame objects.
all_tables = pd.read_html(TOP_30_POLICIES_URL)

# Define Pandas Dataframe
df_rushing_policy = all_tables[0]

# Check number of table(s)
df_rushing_policy.dropna(axis = 0, inplace = True)

new_name = {'เป้าหมายตามนโยบาย ผว.กทม.': 'Goal', 'หน่วยนับ':'Unit', 'เป้าหมายรวม': 'Total_Goals',
    'เป้าหมายจากสำนัก':'Goal_Departments', 'เป้าหมายจาก 50 เขต':'Goal_50_Districts', 'ผลการดำเนินงาน*':'Total_Progress_in_Unit',
    'ความคืบหน้า* ของ KPI (%)':'Progress_in_Percent'}

df_rushing_policy.rename(columns = new_name, inplace = True)

# new data frame with split value columns
splited_goal = df_rushing_policy["Goal"].str.split(":", n=1, expand=True)

# making separate first name column from new data frame
df_rushing_policy["Goal"] = splited_goal[0]

# making separate last name column from new data frame
df_rushing_policy["Related_OKR"] = splited_goal[1]

# Dropping old Name columns
# df_rushing_policy.drop(columns=["Goal"], inplace=True)

df_rushing_policy['Related_OKR'] = df_rushing_policy['Related_OKR'].str.replace('OKR ', '')
df_rushing_policy['Related_OKR'] = df_rushing_policy['Related_OKR'].str.replace('(** ค่าเฉลี่ย **)', '(Mean Value)')

splited_goal_2 = df_rushing_policy["Goal"].str.split(" ", n=1, expand=True)
df_rushing_policy["ID_Result"] = splited_goal_2[0]
df_rushing_policy["Goal"] = splited_goal_2[1]

df_rushing_policy['ID_Result'] = df_rushing_policy['ID_Result'].str.strip('.')

today = datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%d-%m-%Y")
df_rushing_policy['Updated_Date'] = today

df_rushing_policy['Updated_Date'] = df_rushing_policy['Updated_Date'].astype('datetime64[us]')
df_rushing_policy = df_rushing_policy.astype({'ID_Result': float})

df_rushing_policy = df_rushing_policy[['Updated_Date','ID_Result', 'Related_OKR', 'Goal', 'Unit', 'Total_Goals', 'Goal_Departments',
    'Goal_50_Districts', 'Total_Progress_in_Unit', 'Progress_in_Percent']]

In [34]:
df_rushing_policy.head(30)

,Updated_Date,ID_Result,Related_OKR,Goal,Unit,Total_Goals,Goal_Departments,Goal_50_Districts,Total_Progress_in_Unit,Progress_in_Percent
0,2024-12-08,1.0,4.1.10,พัฒนาถนนสวย (กม.),กม.,1.333500e+02,0.00,1.333500e+02,1.284600e+02,96.34
1,2024-12-08,2.0,4.1.01,ปลูกต้นไม้รวมทุกประเภท (ต้น),ต้น,2.000000e+05,50000.00,1.500000e+05,3.629420e+05,181.47
2,2024-12-08,3.0,4.1.05,เพิ่มสวน 15 นาที (แห่ง),แห่ง,1.560000e+02,6.00,1.500000e+02,1.280000e+02,82.05
3,2024-12-08,4.1,1.1.05,ปรับปรุงทางเท้า (กม.),กม.,3.241100e+02,300.00,2.411000e+01,1.110700e+02,34.27
4,2024-12-08,4.2,1.3.07,ทางจักรยาน (กม.),กม.,5.190100e+02,18.00,5.010100e+02,2.261900e+02,43.58
5,2024-12-08,4.3,None,ปรับปรุงซอย (กม.),กม.,1.360000e+02,0.00,1.360000e+02,7.609000e+01,55.95
6,2024-12-08,5.0,2.1.20,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),ดวง,3.207800e+04,13100.00,1.897800e+04,4.058800e+04,126.53
7,2024-12-08,6.0,1.1.03,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),ดวง,8.056000e+03,2114.00,5.942000e+03,4.800000e+03,59.58
8,2024-12-08,7.0,1.1.01,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน...,แห่ง,1.090000e+02,0.00,1.090000e+02,1.150000e+02,105.50
9,2024-12-08,8.0,7.2.05,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รอง...,แห่ง,2.800000e+01,0.00,2.800000e+01,2.600000e+01,92.85


In [35]:
df_rushing_policy.dtypes

,0
Updated_Date,datetime64[us]
ID_Result,float64
Related_OKR,object
Goal,object
Unit,object
Total_Goals,float64
Goal_Departments,float64
Goal_50_Districts,float64
Total_Progress_in_Unit,float64
Progress_in_Percent,float64


In [36]:
df_rushing_policy.to_parquet('top-policy-' + str(today) + '.parquet', index=False)

## BKK: Monthly Progress of Policy

In [ ]:
import pandas as pd
import numpy as np
import re

### Initialized DataFrame

In [ ]:
# Declare Blank DataFrame
df_progress = pd.DataFrame()

# Access and concat all DataFrames to same DataFrames
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    progress_table = pd.read_html(policy_url)
    df_progress = pd.concat([df_progress, progress_table[0]], ignore_index=True, axis = 0)

## Rename Columns
new_col_names = {'KEY_RESULT':'Goal', 'ค่าเป้าหมาย/ปี':'Yearly_Goal', 'ผลดำเนินงาน (รวม)':'Total_Progress_Unit', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}
df_progress.rename(columns = new_col_names, inplace = True)

# Drop Null Value
df_progress.dropna(axis = 0, inplace = True)

# Reset Index
df_progress = df_progress.reset_index()
df_progress.drop('index', axis = 1, inplace = True)

# Set Index
df_progress['#'] = df_progress['#'].astype('int')
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)

In [ ]:
df_progress.head()

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 149.45%,200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 125.81%,31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0


### Extract OKR value

In [ ]:
# Function to extract all occurrences of "OKR \d+\.\d+\.\d+" (OKR 0.0.0)
def extract_okr_numbers(text):
    okrs = re.findall(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', text)
    okrs = ''.join(okrs)
    result = okrs.replace('OKR ', '', )
    return result

def remove_okr_numbers(text):
    return re.sub(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', '', text).strip()

# Apply the function to the Goal column
df_progress['Related_OKRs'] = df_progress['Goal'].apply(extract_okr_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_okr_numbers(x))


# Replace all Blank value by NaN (Null)
df_progress['Related_OKRs'] = df_progress['Related_OKRs'].replace('', np.nan)

### Extract KPI value

In [ ]:
# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_kpi_numbers(text):
    kpi = re.findall(r'KPI \d+(?:\.\d+){2,3}', text)
    kpi = ''.join(kpi)
    result = kpi.replace('KPI ', '', )
    return result

def remove_kpi_numbers(text):
    return re.sub(r'KPI \d+(?:\.\d+){2,3}', '', text).strip()

df_progress['Related_KPI'] = df_progress['Goal'].apply(extract_kpi_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_kpi_numbers(x))

df_progress['Related_KPI'] = df_progress['Related_KPI'].replace('', np.nan)

### Extract Percent Progress

In [ ]:
# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_percent_progress(text):
    progress_percent = re.findall(r'\d{1,3}(?:,\d{3})*\.\d+%', text)
    return ''.join(progress_percent)

def remove_percent_progress(text):
    return re.sub(r'\d{1,3}(?:,\d{3})*\.\d+%', '', text).strip()


# Dealing with Total_Progress_in_Percent
df_progress['Total_Progress_in_Percent'] = df_progress['Goal'].apply(extract_percent_progress)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_percent_progress(x))

df_progress['Goal'] = df_progress['Goal'].str.replace(':', '')

#Check DF info
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   Yearly Goal            300 non-null    float64
 2   Total Progress (Unit)  300 non-null    float64
 3   Unit                   300 non-null    object 
 4   Oct 23                 300 non-null    float64
 5   Nov 23                 300 non-null    float64
 6   Dec 23                 300 non-null    float64
 7   Jan 24                 300 non-null    float64
 8   Feb 24                 300 non-null    float64
 9   Mar 24                 300 non-null    float64
 10  Apr 24                 300 non-null    float64
 11  May 24                 300 non-null    float64
 12  Jun 24                 300 non-null    float64
 13  July 24                300 non-null    float64
 14  Aug 24                 300 non-null    float64
 15  Sept 24    

### Extract Goal_ID

In [ ]:
def extract_goal_id(text):
    goal_id = re.findall(r'\d+\.\d?', str(text))
    result = ''.join(goal_id)
    return result
# def extract_patterns(text):
#     return re.findall(r'\b\d+\.\d*\b', text)

def remove_goal_id(text):
    return re.sub(r'\d+\.\d?', '', text).strip()


df_progress['ID_Result'] = df_progress['Goal'].apply(extract_goal_id)
df_progress['ID_Result'] = df_progress['ID_Result'].replace('', np.nan)
df_progress['ID_Result'] = df_progress['ID_Result'].str.strip('.')

df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_goal_id(x))

In [ ]:
df_progress = df_progress[['Goal','ID_Result','Unit', 'Related_OKRs', 'Related_KPI', 'Yearly_Goal', 'Total_Progress_Unit', 'Total_Progress_in_Percent',
             'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24']]

In [ ]:
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   No. (Goal)             27 non-null     object 
 2   Unit                   300 non-null    object 
 3   Related OKRs           58 non-null     object 
 4   Related KPI            254 non-null    object 
 5   Yearly Goal            300 non-null    float64
 6   Total Progress (Unit)  300 non-null    float64
 7   Total Progress (%)     300 non-null    object 
 8   Oct 23                 300 non-null    float64
 9   Nov 23                 300 non-null    float64
 10  Dec 23                 300 non-null    float64
 11  Jan 24                 300 non-null    float64
 12  Feb 24                 300 non-null    float64
 13  Mar 24                 300 non-null    float64
 14  Apr 24                 300 non-null    float64
 15  May 24     

In [ ]:
df_progress.head(40)

,Goal,No. (Goal),Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,พัฒนาถนนสวย (กม.),1,กม.,4.1.10,NaN,1.236000e+02,1.271500e+02,102.87%,2.27,0.20,3.49,17.01,20.46,26.27,17.60,12.39,9.15,18.32,0.00,0.0
2,ปลูกต้นไม้รวมทุกประเภท (ต้น),2,ต้น,4.1.01,NaN,2.000000e+05,3.597070e+05,179.85%,8821.00,24435.00,15020.00,29573.00,38701.00,51849.00,29556.00,25802.00,36120.00,93926.00,5904.00,0.0
3,เพิ่มสวน 15 นาที (แห่ง),3,แห่ง,4.1.05,NaN,1.560000e+02,1.290000e+02,82.69%,2.00,6.00,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,3.00,0.0
4,ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,3.120700e+02,1.154700e+02,37.00%,0.00,0.00,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.24,0.0
5,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5,ดวง,2.1.20,NaN,3.187800e+04,4.089400e+04,128.28%,510.00,3578.00,3860.00,5048.00,3860.00,6707.00,3924.00,4371.00,3653.00,4736.00,647.00,0.0
6,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6,ดวง,1.1.03,NaN,9.536000e+03,4.815000e+03,50.49%,70.00,316.00,234.00,325.00,1413.00,345.00,506.00,471.00,416.00,623.00,96.00,0.0
7,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน...,7,แห่ง,1.1.01,NaN,1.000000e+02,1.170000e+02,117.00%,3.00,1.00,1.00,12.00,14.00,30.00,23.00,12.00,7.00,11.00,3.00,0.0
8,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รอง...,8,แห่ง,7.2.05,NaN,1.100000e+01,3.000000e+01,272.73%,1.00,3.00,1.00,4.00,2.00,7.00,4.00,4.00,1.00,2.00,1.00,0.0
9,แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9,จุด,1.4.05,NaN,2.120000e+02,1.160000e+02,54.72%,5.00,22.00,1.00,5.00,8.01,32.00,12.20,15.20,5.45,10.14,0.00,0.0


In [ ]:
df_progress.to_csv('all_progress.csv')

In [ ]:
df_progress.to_parquet('all_progress.parquet')

### All code of extracting and transform all policy

In [5]:
import re
import numpy as np
import pandas as pd
from datetime import datetime
import pytz

From VS Code

In [72]:
# Declare Blank DataFrame
df_progress = pd.DataFrame()

# Access and concat all DataFrames to same DataFrames
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    progress_table = pd.read_html(policy_url)
    df_progress = pd.concat([df_progress, progress_table[0]], ignore_index=True, axis = 0)

## Rename Columns
new_col_names = {'KEY_RESULT':'Goal', 'ค่าเป้าหมาย/ปี':'Yearly_Goal', 'ผลดำเนินงาน (รวม)':'Total_Progress_in_Unit', 'หน่วยนับ':'Unit',
    'ตค.66':'Oct_23', 'พย.66':'Nov_23', 'ธค.66':'Dec_23', 'มค.67':'Jan_24', 'กพ.67':'Feb_24', 'มี.ค.67':'Mar_24', 'เม.ย.67':'Apr_24',
    'พค.67':'May_24', 'มิ.ย.67':'Jun_24', 'กค.67':'July_24', 'สค.67':'Aug_24', 'กย.67':'Sept_24'}
df_progress.rename(columns = new_col_names, inplace = True)

# Drop Null Value
df_progress.dropna(axis = 0, inplace = True)

# Reset Index
df_progress = df_progress.reset_index()
df_progress.drop('index', axis = 1, inplace = True)

# Set Index
df_progress['#'] = df_progress['#'].astype('int')
df_progress.set_index(df_progress['#'], inplace = True)
#df_progress.drop('#', axis = 1, inplace = True)

# Function to extract all occurrences of "OKR \d+\.\d+\.\d+" (OKR 0.0.0)
def extract_okr_numbers(text):
    okrs = re.findall(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', text)
    okrs = ''.join(okrs)
    result = okrs.replace('OKR ', '', )
    return result

def remove_okr_numbers(text):
    return re.sub(r'OKR \d+\.\d+\.\d+(?:\.\d+)?', '', text).strip()

# Apply the function to the Goal column
df_progress['Related_OKRs'] = df_progress['Goal'].apply(extract_okr_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_okr_numbers(x))


# Replace all Blank value by NaN (Null)
df_progress['Related_OKRs'] = df_progress['Related_OKRs'].replace('', np.nan)

# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_kpi_numbers(text):
    kpi = re.findall(r'KPI \d+(?:\.\d+){2,3}', text)
    kpi = ''.join(kpi)
    result = kpi.replace('KPI ', '', )
    return result

def remove_kpi_numbers(text):
    return re.sub(r'KPI \d+(?:\.\d+){2,3}', '', text).strip()

df_progress['Related_KPI'] = df_progress['Goal'].apply(extract_kpi_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_kpi_numbers(x))

df_progress['Related_KPI'] = df_progress['Related_KPI'].replace('', np.nan)

# Function to extract all occurrences of "\d+\.\d+%" (xxx.xx%)
def extract_percent_progress(text):
    progress_percent = re.findall(r'\d{1,3}(?:,\d{3})*\.\d+%', text)
    return ''.join(progress_percent)

def remove_percent_progress(text):
    return re.sub(r'\d{1,3}(?:,\d{3})*\.\d+%', '', text).strip()


# Dealing with Total_Progress_in_Percent
df_progress['Total_Progress_in_Percent'] = df_progress['Goal'].apply(extract_percent_progress)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_percent_progress(x))

df_progress['Goal'] = df_progress['Goal'].str.replace(':', '')

def extract_goal_id(text):
    goal_id = re.findall(r'\d+\.\d?', str(text))
    result = ''.join(goal_id)
    return result
# def extract_patterns(text):
#     return re.findall(r'\b\d+\.\d*\b', text)

def remove_goal_id(text):
    return re.sub(r'\d+\.\d?', '', text).strip()

df_progress['ID_Result'] = df_progress['Goal'].apply(extract_goal_id)
df_progress['ID_Result'] = df_progress['ID_Result'].replace('', np.nan)
df_progress['ID_Result'] = df_progress['ID_Result'].str.strip('.')

df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_goal_id(x))

# Create Updated_Date columns to store the date
today = datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%d-%m-%Y")
df_progress['Updated_Date'] = today

# Convert to datetime64[us] data type
df_progress['Updated_Date'] = df_progress['Updated_Date'].astype('datetime64[us]')
df_progress = df_progress.astype({'ID_Result': float})

df_progress = df_progress[['Updated_Date','#','Goal','ID_Result','Unit', 'Related_OKRs', 'Related_KPI', 'Yearly_Goal', 'Total_Progress_in_Unit', 'Total_Progress_in_Percent',
            'Oct_23', 'Nov_23', 'Dec_23', 'Jan_24', 'Feb_24', 'Mar_24', 'Apr_24', 'May_24', 'Jun_24', 'July_24', 'Aug_24', 'Sept_24']]

In [73]:
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Updated_Date               300 non-null    datetime64[us]
 1   #                          300 non-null    int64         
 2   Goal                       300 non-null    object        
 3   ID_Result                  27 non-null     float64       
 4   Unit                       300 non-null    object        
 5   Related_OKRs               57 non-null     object        
 6   Related_KPI                255 non-null    object        
 7   Yearly_Goal                300 non-null    float64       
 8   Total_Progress_in_Unit     300 non-null    float64       
 9   Total_Progress_in_Percent  300 non-null    object        
 10  Oct_23                     300 non-null    float64       
 11  Nov_23                     300 non-null    float64       
 12  Dec_23       

In [67]:
df_progress['Oct_23_Percent'] = df_progress['Oct_23'].div(df_progress.Yearly_Goal, axis=0)*100
df_progress['Oct_23_Percent'] = df_progress['Oct_23_Percent'].round(2)

In [68]:
df_progress[['#', 'Goal', 'Yearly_Goal', 'Oct_23', 'Oct_23_Percent']].head(30)

,#,Goal,Yearly_Goal,Oct_23,Oct_23_Percent
#,,,,,
1,1,พัฒนาถนนสวย (กม.),123.6,2.27,1.84
2,2,ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,8821.0,4.41
3,3,เพิ่มสวน 15 นาที (แห่ง),156.0,2.0,1.28
4,4,ปรับปรุงทางเท้า (กม.),312.07,0.0,0.0
5,5,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,510.0,1.6
6,6,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),9536.0,70.0,0.73
7,7,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),100.0,3.0,3.0
8,8,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),11.0,1.0,9.09
9,9,แก้ไขจุดเสี่ยงน้ำท่วม (จุด),212.0,5.0,2.36


In [58]:
df_progress.head(30)

,Updated_Date,#,Goal,ID_Result,Unit,Related_OKRs,Related_KPI,Yearly_Goal,Total_Progress_in_Unit,Total_Progress_in_Percent,...,Dec_23,Jan_24,Feb_24,Mar_24,Apr_24,May_24,Jun_24,July_24,Aug_24,Sept_24
#,,,,,,,,,,,,,,,,,,,,,
1,2024-12-08,1,พัฒนาถนนสวย (กม.),1.0,กม.,4.1.10,NaN,1.236000e+02,1.290500e+02,104.41%,...,3.49,17.01,20.46,26.27,17.60,12.39,9.15,20.22,0.00,0.0
2,2024-12-08,2,ปลูกต้นไม้รวมทุกประเภท (ต้น),2.0,ต้น,4.1.01,NaN,2.000000e+05,3.629420e+05,181.47%,...,15020.00,29573.00,38701.00,51849.00,29556.00,25902.00,36020.00,95836.00,7229.00,0.0
3,2024-12-08,3,เพิ่มสวน 15 นาที (แห่ง),3.0,แห่ง,4.1.05,NaN,1.560000e+02,1.300000e+02,83.33%,...,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,4.00,0.0
4,2024-12-08,4,ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,3.120700e+02,1.158700e+02,37.13%,...,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.64,0.0
5,2024-12-08,5,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5.0,ดวง,2.1.20,NaN,3.187800e+04,4.089400e+04,128.28%,...,3860.00,5048.00,3860.00,6707.00,3924.00,4371.00,3653.00,4736.00,647.00,0.0
6,2024-12-08,6,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6.0,ดวง,1.1.03,NaN,9.536000e+03,4.815000e+03,50.49%,...,234.00,325.00,1413.00,345.00,506.00,471.00,416.00,623.00,96.00,0.0
7,2024-12-08,7,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน...,7.0,แห่ง,1.1.01,NaN,1.000000e+02,1.170000e+02,117.00%,...,1.00,12.00,14.00,30.00,23.00,12.00,7.00,11.00,3.00,0.0
8,2024-12-08,8,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รอง...,8.0,แห่ง,7.2.05,NaN,1.100000e+01,3.000000e+01,272.73%,...,1.00,4.00,2.00,7.00,4.00,4.00,1.00,2.00,1.00,0.0
9,2024-12-08,9,แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9.0,จุด,1.4.05,NaN,2.120000e+02,1.160000e+02,54.72%,...,1.00,5.00,8.01,32.00,12.20,15.20,5.45,10.14,0.00,0.0


In [ ]:
df_progress.head(30)

,Goal,No. (Goal),Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,พัฒนาถนนสวย (กม.),1,กม.,4.1.10,NaN,1.236000e+02,1.290500e+02,104.41%,2.27,0.20,3.49,17.01,20.46,26.27,17.60,12.39,9.15,20.22,0.00,0.0
2,ปลูกต้นไม้รวมทุกประเภท (ต้น),2,ต้น,4.1.01,NaN,2.000000e+05,3.629420e+05,181.47%,8821.00,24435.00,15020.00,29573.00,38701.00,51849.00,29556.00,25902.00,36020.00,95836.00,7229.00,0.0
3,เพิ่มสวน 15 นาที (แห่ง),3,แห่ง,4.1.05,NaN,1.560000e+02,1.300000e+02,83.33%,2.00,6.00,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,4.00,0.0
4,ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,3.120700e+02,1.154700e+02,37.00%,0.00,0.00,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.24,0.0
5,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5,ดวง,2.1.20,NaN,3.187800e+04,4.089400e+04,128.28%,510.00,3578.00,3860.00,5048.00,3860.00,6707.00,3924.00,4371.00,3653.00,4736.00,647.00,0.0
6,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6,ดวง,1.1.03,NaN,9.536000e+03,4.815000e+03,50.49%,70.00,316.00,234.00,325.00,1413.00,345.00,506.00,471.00,416.00,623.00,96.00,0.0
7,ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน...,7,แห่ง,1.1.01,NaN,1.000000e+02,1.170000e+02,117.00%,3.00,1.00,1.00,12.00,14.00,30.00,23.00,12.00,7.00,11.00,3.00,0.0
8,จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รอง...,8,แห่ง,7.2.05,NaN,1.100000e+01,3.000000e+01,272.73%,1.00,3.00,1.00,4.00,2.00,7.00,4.00,4.00,1.00,2.00,1.00,0.0
9,แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9,จุด,1.4.05,NaN,2.120000e+02,1.160000e+02,54.72%,5.00,22.00,1.00,5.00,8.01,32.00,12.20,15.20,5.45,10.14,0.00,0.0


# Joining df_rushing_policy to df_progress

## Read relevant columns from the two tables

In [ ]:
df_rushing_policy.head()

,No. (Goal),Related OKR,Goal,Unit,Total Goals,Goal (Departments),Goal (50 Districts),Total Progress (Unit),Progress (%)
0,1,4.1.10,พัฒนาถนนสวย (กม.),กม.,133.35,0.0,133.35,128.46,96.34
1,2,4.1.01,ปลูกต้นไม้รวมทุกประเภท (ต้น),ต้น,200000.00,50000.0,150000.00,362942.00,181.47
2,3,4.1.05,เพิ่มสวน 15 นาที (แห่ง),แห่ง,156.00,6.0,150.00,128.00,82.05
3,4.1,1.1.05,ปรับปรุงทางเท้า (กม.),กม.,324.11,300.0,24.11,110.67,34.15
4,4.2,1.3.07,ทางจักรยาน (กม.),กม.,519.01,18.0,501.01,226.19,43.58


In [ ]:
df_progress.head()

,Goal,No. (Goal),Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,พัฒนาถนนสวย (กม.),1,กม.,4.1.10,NaN,123.60,129.05,104.41%,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,9.15,20.22,0.00,0.0
2,ปลูกต้นไม้รวมทุกประเภท (ต้น),2,ต้น,4.1.01,NaN,200000.00,362942.00,181.47%,8821.00,24435.0,15020.00,29573.00,38701.00,51849.00,29556.00,25902.00,36020.00,95836.00,7229.00,0.0
3,เพิ่มสวน 15 นาที (แห่ง),3,แห่ง,4.1.05,NaN,156.00,130.00,83.33%,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,4.00,0.0
4,ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,312.07,115.47,37.00%,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.24,0.0
5,ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5,ดวง,2.1.20,NaN,31878.00,40894.00,128.28%,510.00,3578.0,3860.00,5048.00,3860.00,6707.00,3924.00,4371.00,3653.00,4736.00,647.00,0.0


In [ ]:
df_progress.columns

Index(['Goal', 'No. (Goal)', 'Unit', 'Related OKRs', 'Related KPI',
       'Yearly Goal', 'Total Progress (Unit)', 'Total Progress (%)', 'Oct 23',
       'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24',
       'Jun 24', 'July 24', 'Aug 24', 'Sept 24'],
      dtype='object')

In [ ]:
df_rushing_policy_for_join = pd.read_parquet('df_rushing_policies.parquet', columns=['ID_Result','Goal'])
df_progress_for_join = pd.read_parquet('all_progress.parquet', columns = ['ID_Result', 'Yearly_Goal', 'Total_Progress_Unit', 'Unit', 'Total_Progress_in_Percent', 'Oct 23',
       'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24'])

In [ ]:
df_joined = df_rushing_policy_for_join.merge(df_progress_for_join, left_on = 'ID_Result', right_on = 'ID_Result', how = 'left')

In [ ]:
df_joined.columns

Index(['No. (Goal)', 'Goal', 'Unit', 'Yearly Goal', 'Total Progress (Unit)',
       'Total Progress (%)', 'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24',
       'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24'],
      dtype='object')

In [ ]:
df_joined = df_joined[['ID_Result', 'Goal', 'Yearly_Goal', 'Total_Progress_Unit', 'Unit',
       'Total_Progress_in_Percent', 'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24',
       'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24']]

In [ ]:
df_joined.head()

,No. (Goal),Goal,Yearly Goal,Total Progress (Unit),Unit,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
0,1,พัฒนาถนนสวย (กม.),123.60,129.05,กม.,104.41%,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,9.15,20.22,0.00,0.0
1,2,ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.00,362942.00,ต้น,181.47%,8821.00,24435.0,15020.00,29573.00,38701.00,51849.00,29556.00,25902.00,36020.00,95836.00,7229.00,0.0
2,3,เพิ่มสวน 15 นาที (แห่ง),156.00,130.00,แห่ง,83.33%,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,4.00,0.0
3,4.1,ปรับปรุงทางเท้า (กม.),312.07,115.47,กม.,37.00%,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.24,0.0
4,4.2,ทางจักรยาน (กม.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_joined.columns

Index(['No. (Goal)', 'Goal', 'Yearly Goal', 'Total Progress (Unit)', 'Unit',
       'Total Progress (%)', 'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24',
       'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24'],
      dtype='object')

In [ ]:
df_joined.loc[df_joined['Feb 24'].isna()]

,No. (Goal),Goal,Yearly Goal,Total Progress (Unit),Unit,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
4,4.2,ทางจักรยาน (กม.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.3,ปรับปรุงซอย (กม.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,10.2,การผายปากซอย (จุด),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,11.3,จำนวนมอเตอร์ไซต์วิ่งบนทางเท้าที่จับได้เพิ่มขึ้...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,16.3,การตั้งจุดทิ้งขยะกลางในหมู่บ้าน (จุด),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,23,จำนวนรายงานผลการดำเนินงานของคณะขับเคลื่อนการมี...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_joined.head(30)

,No. (Goal),Goal_All,Related OKR,Total Progress (Unit)_All,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
0,1,1. พัฒนาถนนสวย (กม.),4.1.10,126.85,102.63%,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,9.15,18.02,0.0,0.0
1,2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),4.1.01,301839.0,150.92%,8734.0,24454.0,14792.0,29427.0,38556.0,51876.0,29474.0,25720.0,36038.0,39084.0,3684.0,0.0
2,3,3. เพิ่มสวน 15 นาที (แห่ง),4.1.05,128.0,82.05%,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,20.0,20.0,2.0,0.0
3,4.1,4.1 ปรับปรุงทางเท้า (กม.),1.1.05,114.19,36.59%,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.73,94.09,0.16,0.0
4,4.2,NaN,1.3.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),2.1.20,40661.0,127.55%,510.0,3578.0,3860.0,5048.0,3860.0,6707.0,3924.0,4371.0,3630.0,4732.0,441.0,0.0
7,6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),1.1.03,4721.28,49.51%,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,623.28,0.0,0.0
8,7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),1.1.01,115.0,115.00%,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,1.0,0.0
9,8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),7.2.05,30.0,272.73%,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,1.0,0.0


In [ ]:
df_progress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 300
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Goal                   300 non-null    object 
 1   Yearly Goal            300 non-null    float64
 2   Total Progress (Unit)  300 non-null    float64
 3   Related OKRs           58 non-null     object 
 4   Related KPI            254 non-null    object 
 5   Total Progress (%)     300 non-null    object 
 6   Unit                   300 non-null    object 
 7   Oct 23                 300 non-null    float64
 8   Nov 23                 300 non-null    float64
 9   Dec 23                 300 non-null    float64
 10  Jan 24                 300 non-null    float64
 11  Feb 24                 300 non-null    float64
 12  Mar 24                 300 non-null    float64
 13  Apr 24                 300 non-null    float64
 14  May 24                 300 non-null    float64
 15  Jun 24     

In [ ]:
df_progress.head(50)

,Goal,No. (Goal),Unit,Related OKRs,Related KPI,Yearly Goal,Total Progress (Unit),Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),1,กม.,4.1.10,NaN,123.6,126.85,102.63%,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,9.15,18.02,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),2,ต้น,4.1.01,NaN,200000.0,301839.0,150.92%,8734.0,24454.0,14792.0,29427.0,38556.0,51876.0,29474.0,25720.0,36038.0,39084.0,3684.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง),3,แห่ง,4.1.05,NaN,156.0,128.0,82.05%,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,20.0,20.0,2.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.),4.1,กม.,1.1.05,NaN,312.07,114.19,36.59%,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.73,94.09,0.16,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),5,ดวง,2.1.20,NaN,31878.0,40661.0,127.55%,510.0,3578.0,3860.0,5048.0,3860.0,6707.0,3924.0,4371.0,3630.0,4732.0,441.0,0.0
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),6,ดวง,1.1.03,NaN,9536.0,4721.28,49.51%,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,623.28,0.0,0.0
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),7,แห่ง,1.1.01,NaN,100.0,115.0,115.00%,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,1.0,0.0
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),8,แห่ง,7.2.05,NaN,11.0,30.0,272.73%,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,1.0,0.0
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),9,จุด,1.4.05,NaN,212.0,223.0,105.19%,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,113.14,0.0,0.0


In [ ]:
df_progress.head(40)

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Total Progress (%),Related KPI
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,4.1.10,105.05%,NaN
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,4.1.01,149.45%,NaN
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,4.1.05,79.49%,NaN
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,1.1.05,36.44%,NaN
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,2.1.20,125.81%,NaN
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),9536.0,4701.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,603.28,0.0,0.0,1.1.03,49.30%,NaN
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),100.0,114.0,แห่ง,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0,1.1.01,114.00%,NaN
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),11.0,29.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,0.0,0.0,7.2.05,263.64%,NaN
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),212.0,222.0,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,112.14,0.0,0.0,1.4.05,104.72%,NaN


In [ ]:
df_progress.loc[df_progress['Related_KPI'].isna() == False]

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Related KPI,Total Progress (%)
#,,,,,,,,,,,,,,,,,,,
26,ความสำเร็จในการแก้ไขปัญหาข้อร้องเรียนของประชาชนผ่าน Traffy Fondue,90.0,17243.72,ร้อยละ,591.76,1484.03,1723.35,1918.12,2073.35,1868.41,1934.12,2483.8,1406.42,1760.36,0.0,0.0,NaN,.1,"19,159.69%"
27,เรื่องที่ได้รับการแก้ไข ปรับปรุง หรือส่งต่อหน่วยงานเจ้าของภารกิจ,100.0,142.0,ร้อยละ,10.0,0.0,0.0,0.0,0.0,0.0,50.0,10.0,72.0,0.0,0.0,0.0,NaN,.1,142.00%
28,จำนวนบริการหรือกิจกรรมสาธารณะของ กรุงเทพมหานคร ที่ประชาชนหรือภาคส่วนอื่น ๆ เข้ามามีส่วนร่วม,1.0,109.7,บริการต่อกิจกรรมสาธารณะ,0.0,0.0,0.0,15.0,94.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.3,"10,970.00%"
29,สำนักงานเขตจัดทำแผนพัฒนาเขตที่มาจากการมีส่วนร่วมของหน่วยนโยบาย ชุมชน และภาคประชาชน และให้กรุงเทพมหานครนำแผนส่วนนี้เข้ามาพิจารณาเพื่อจัดสรรงบประมาณให้กับสำนักงานเขต,50.0,0.0,โครงการ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.1,0.00%
30,"โครงการที่มาจากกระบวนการมีส่วนร่วมของประชาชนได้รับการบรรจุ ในแผน 2,017 ชุมชน และดำเนินการ",2017.0,163.0,ชุมชน,0.0,9.0,9.0,8.0,14.0,85.0,9.0,8.0,11.0,10.0,0.0,0.0,NaN,.2,8.08%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,กลุ่มเปราะบางที่ได้รับการจ้างงานจากหน่วยงานในสังกัดกรุงเทพมหานคร (เช่น คนพิการ ผู้สูงอายุ คนไร้บ้าน แม่ที่ท้องไม่พร้อม ฯลฯ),100.0,2684.33,ร้อยละ,639.33,259.0,263.0,160.5,173.5,253.5,251.83,162.0,270.67,251.0,0.0,0.0,NaN,.1,"2,684.33%"
276,กลุ่มเปราะบางที่เข้ารับการฝึกอาชีพที่โรงเรียน/ศูนย์ฝึกอาชีพสังกัดกรุงเทพมหานคร และนำไปประกอบอาชีพสร้างรายได้จริง (เช่น คนพิการ ผู้สูงอายุ แรงงานนอกระบบ ฯลฯ),50.0,0.0,ร้อยละ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,.2,0.00%
277,ข้าราชการ และบุคลากรของกรุงเทพมหานครที่ได้รับการอบรม ฝึกทักษะ เพื่อทำงานร่วมกับคนกลุ่มเปราะบางกลุ่มต่าง ๆ,100.0,300.0,ร้อยละ,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,NaN,.3,300.00%


In [ ]:
df_progress

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Related KPI,Total Progress (%)
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,NaN,NaN,
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,NaN,NaN,
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,NaN,NaN,
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,NaN,NaN,
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,
297,"จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,
298,ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,


In [ ]:
df_progress.isnull().sum()

,0
Key_Result,0
Yearly Goal,0
Acheived Goal,0
Unit,0
Oct 23,0
Nov 23,0
Dec 23,0
Jan 24,0
Feb 24,0
Mar 24,0


In [ ]:
# # Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('OKR')]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,1.236000e+02,1.298400e+02,กม.,2.27,0.20,3.49,17.01,20.46,26.27,17.60,12.39,7.65,22.51,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,2.000000e+05,2.988750e+05,ต้น,8734.00,24454.00,14886.00,29427.00,38556.00,51876.00,29415.00,25710.00,35950.00,39867.00,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,1.560000e+02,1.240000e+02,แห่ง,2.00,6.00,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,3.120700e+02,1.137300e+02,กม.,0.00,0.00,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,3.187800e+04,4.005500e+04,ดวง,2103.00,3578.00,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3368.00,0.0,0.0
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : ...,9.536000e+03,4.657280e+03,ดวง,70.00,316.00,234.00,325.00,1413.00,345.00,507.00,472.00,416.00,559.28,0.0,0.0
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อน...,1.000000e+02,1.140000e+02,แห่ง,3.00,1.00,1.00,12.00,14.00,30.00,23.00,12.00,7.00,11.00,0.0,0.0
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center ...,1.100000e+01,2.900000e+01,แห่ง,1.00,3.00,1.00,4.00,2.00,7.00,4.00,4.00,1.00,2.00,0.0,0.0
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด) : OKR 1.4.05 1...,2.120000e+02,2.220000e+02,จุด,5.00,22.00,1.00,5.00,8.01,32.00,12.20,17.20,7.45,112.14,0.0,0.0


In [ ]:
# Function to extract all occurrences of "OKR \d+\.\d+\.\d+"
def extract_okr_numbers(text):
    okrs = re.findall(r'OKR (\d+\.\d+\.\d+)', text)
    return ''.join(okrs)

def remove_okr_numbers(text):
    return re.sub(r'OKR \d+\.\d+\.\d+', '', text).strip()

def extract_percent_progress(text):
    progress_percent = re.findall(r'\d+\.\d+%', text)
    return ''.join(progress_percent)

def remove_percent_progress(text):
    return re.sub(r'\d+\.\d+%', '', text).strip()

def remove_semi_colon(text):
    return re.sub(r'\d+\.\d+%', '', text).strip()


# Apply the function to the Goal column
df_progress['Extracted_OKRs'] = df_progress['Goal'].apply(extract_okr_numbers)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_okr_numbers(x))

df_progress['Extracted_OKRs'] = df_progress['Extracted_OKRs'].replace(r'^\s*$', np.nan, regex=True)

df_progress['Total_Progress_in_Percent'] = df_progress['Goal'].apply(extract_percent_progress)
df_progress['Goal'] = df_progress['Goal'].apply(lambda x: remove_percent_progress(x))

df_progress['Total_Progress_in_Percent'] = df_progress['Total_Progress_in_Percent'].replace(r'^\s*$', np.nan, regex=True)

df_progress['Goal'].str.replace(':', '')

df_progress.info()

KeyError: 'Goal'

In [ ]:
df_progress.head(10)

,Goal,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Related OKRs,Total Progress (%),Extracted_OKRs
#,,,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.),123.6,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,22.51,0.0,0.0,NaN,NaN,NaN
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.0,298905.0,ต้น,8734.0,24454.0,14886.0,29427.0,38556.0,51876.0,29415.0,25710.0,35950.0,39897.0,0.0,0.0,NaN,NaN,NaN
3,3. เพิ่มสวน 15 นาที (แห่ง),156.0,124.0,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,24.0,0.0,0.0,NaN,NaN,NaN
4,4.1 ปรับปรุงทางเท้า (กม.),312.07,113.73,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,NaN,NaN,NaN
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง),31878.0,40105.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,3418.0,0.0,0.0,NaN,NaN,NaN
6,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง),9536.0,4701.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,603.28,0.0,0.0,NaN,NaN,NaN
7,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง),100.0,114.0,แห่ง,3.0,1.0,1.0,12.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0,NaN,NaN,NaN
8,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง),11.0,29.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,4.0,4.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN
9,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด),212.0,222.0,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,112.14,0.0,0.0,NaN,NaN,NaN


In [ ]:
df_progress.to_csv("check.csv")

In [ ]:
df_progress[['Extracted_OKRs']].to_csv('okr_list.csv')

In [ ]:
df_progress['Extracted_OKRs'] = df_progress['Extracted_OKRs'].replace(r'^\s*$', np.nan, regex=True)

In [ ]:
df_progress[['Extracted_OKRs']].isna().sum()

,0
Extracted_OKRs,242


In [ ]:
df_progress[['Extracted_OKRs']].isna().sum()

,0
Extracted_OKRs,0


In [ ]:
(df_progress['Extracted_OKRs'].str.strip() == '').sum()

242

In [ ]:
# Contain OKR, but not Contain KPI
df_progress.loc[df_progress['Goal'].str.contains('OKR') & (df_progress['Goal'].str.contains('KPI') == False)].shape

(45, 16)

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24,Extracted_OKRs
#,,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 105.05%,123.60,129.84,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,7.65,22.51,0.0,0.0,4.1.10
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,200000.00,298875.00,ต้น,8734.00,24454.0,14886.00,29427.00,38556.00,51876.00,29415.00,25710.00,35950.00,39867.00,0.0,0.0,4.1.01
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.00,124.00,แห่ง,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0,4.1.05
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.44%,312.07,113.73,กม.,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.89,0.0,0.0,1.1.05
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,31878.00,40055.00,ดวง,2103.00,3578.0,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3368.00,0.0,0.0,2.1.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป...,1.00,1.00,หลักสูตร,0.00,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0,4.2.27
297,OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถ...,2250.00,268.00,กรง (สะสม),0.00,0.0,0.00,0.00,0.00,150.00,118.00,0.00,0.00,0.00,0.0,0.0,4.2.12
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร ...,5000.00,0.00,ชุด,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,4.2.13


In [ ]:
df_progress.loc[df_progress['Goal'].str.contains('OKR')].shape

(58, 16)

In [ ]:
# Contain KPI, but not Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('KPI') & (df_progress['Goal'].str.contains('OKR') == False)]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
26,KPI 5.1.2.1 : ความสำเร็จในการแก้ไขปัญหาข้อร้อง...,90.0,17025.61,ร้อยละ,591.76,1484.03,1723.35,1918.12,2073.35,1868.41,1934.12,2483.8,1406.42,1542.25,0.0,0.0
27,KPI 5.2.1.1 : เรื่องที่ได้รับการแก้ไข ปรับปรุง...,100.0,142.00,ร้อยละ,10.00,0.00,0.00,0.00,0.00,0.00,50.00,10.0,72.00,0.00,0.0,0.0
28,KPI 5.2.1.3 : จำนวนบริการหรือกิจกรรมสาธารณะของ...,1.0,109.70,บริการต่อกิจกรรมสาธารณะ,0.00,0.00,0.00,15.00,94.70,0.00,0.00,0.0,0.00,0.00,0.0,0.0
29,KPI 5.2.2.1 : สำนักงานเขตจัดทำแผนพัฒนาเขตที่มา...,50.0,0.00,โครงการ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
30,KPI 5.2.2.2 : โครงการที่มาจากกระบวนการมีส่วนร่...,2017.0,163.00,ชุมชน,0.00,9.00,9.00,8.00,14.00,85.00,9.00,8.0,11.00,10.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,KPI 3.1.2.1 : กลุ่มเปราะบางที่ได้รับการจ้างงาน...,100.0,2684.33,ร้อยละ,639.33,259.00,263.00,160.50,173.50,253.50,251.83,162.0,270.67,251.00,0.0,0.0
276,KPI 3.1.2.2 : กลุ่มเปราะบางที่เข้ารับการฝึกอาช...,50.0,0.00,ร้อยละ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
277,KPI 3.1.2.3 : ข้าราชการ และบุคลากรของกรุงเทพมห...,100.0,300.00,ร้อยละ,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.0,100.00,0.00,0.0,0.0


In [ ]:
# Not Contain OKR & KPI

df_progress.loc[(df_progress['Goal'].str.contains('KPI') | df_progress['Goal'].str.contains('OKR')) == False]

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
25,"อื่นๆ (ตัวชี้วัดงานประจำ) 166,748.31%",100.0,166748.31,ร้อยละ,4892.96,10198.65,9526.8,11620.32,68166.1,14361.2,11444.29,12508.83,11697.61,12331.55,0.0,0.0


In [ ]:
# # Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('KPI') & (df_progress['Goal'].str.contains('OKR') == False)].shape[0] + \ ## Contain KPI, but not Contain OKR
df_progress.loc[df_progress['Goal'].str.contains('OKR')].shape[0] + \ ##
df_progress.loc[(df_progress['Goal'].str.contains('KPI') | df_progress['Goal'].str.contains('OKR')) == False].shape[0]

300

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 104.01%,123.60,128.55,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,7.65,21.22,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 ...,200000.00,297256.00,ต้น,8734.00,24454.0,14886.00,29427.00,38176.00,51876.00,29415.00,25710.00,35950.00,38628.00,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 79.49%,156.00,124.00,แห่ง,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,14.00,24.00,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.38%,312.07,113.53,กม.,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,3.63,93.69,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1...,31878.00,39947.00,ดวง,2103.00,3578.0,4445.00,4341.00,3876.00,6195.00,3985.00,4336.00,3828.00,3260.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป...,1.00,1.00,หลักสูตร,0.00,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0
297,OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถ...,2250.00,268.00,กรง (สะสม),0.00,0.0,0.00,0.00,0.00,150.00,118.00,0.00,0.00,0.00,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร ...,5000.00,0.00,ชุด,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [ ]:
new_col_names = {'Goal':'Goal', 'ค่าเป้าหมาย/ปี':'Yearly_Goal', 'ผลดำเนินงาน (รวม)':'Acheived Goal', 'หน่วยนับ':'Unit',
       'ตค.66':'Oct 23', 'พย.66':'Nov 23', 'ธค.66':'Dec 23', 'มค.67':'Jan 24', 'กพ.67':'Feb 24', 'มี.ค.67':'Mar 24', 'เม.ย.67':'Apr 24',
       'พค.67':'May 24', 'มิ.ย.67':'Jun 24', 'กค.67':'July 24', 'สค.67':'Aug 24', 'กย.67':'Sept 24'}

In [ ]:
df_progress.rename(columns = new_col_names, inplace = True)

In [ ]:
df_progress

,Key_Result,Yearly Goal,Acheived Goal,Unit,Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.set_index(df_progress['#'], inplace = True)
df_progress.drop('#', axis = 1, inplace = True)
df_progress.head()

,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
#,,,,,,,,,,,,,,,,
1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [ ]:
df_progress.columns

Index(['#', 'KEY_RESULT', 'ค่าเป้าหมาย/ปี', 'ผลดำเนินงาน (รวม)', 'หน่วยนับ',
       'ตค.66', 'พย.66', 'ธค.66', 'มค.67', 'กพ.67', 'มี.ค.67', 'เม.ย.67',
       'พค.67', 'มิ.ย.67', 'กค.67', 'สค.67', 'กย.67'],
      dtype='object')

In [ ]:
df_progress

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,297,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,298,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,299,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.reset_index()

,index,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,300,296.0,OKR 4.2.27 จำนวนหลักสูตรดัดแปลงรถจักรยานยนต์เป็นเครื่องยนต์ไฟฟ้า 100.00%,1.0,1.0,หลักสูตร,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296,301,297.0,"OKR 4.2.12 จำนวนกรงตาข่ายครอบถุงขยะที่ติดตั้งถนนสายหลัก 10,000 กรง 11.91%",2250.0,268.0,กรง (สะสม),0.0,0.0,0.0,0.0,0.0,150.0,118.0,0.0,0.0,0.0,0.0,0.0
297,302,298.0,OKR 4.2.13 ติดตั้งถังขยะ 3 ประเภทขนาด 85 ลิตร (Street Furniture) สำหรับคนเดินเท้าบน ทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,5000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,303,299.0,OKR 4.2.14 ติดตั้งถังขยะ 4 ประเภทขนาด 130 ลิตร (Street Furniture) สำหรับคนเดินเท้าบนทางเท้าถนนสายหลัก ถนนสายรอง 0.00%,10000.0,0.0,ชุด,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_progress.head()

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0


In [ ]:
for i in range(1,7):
    policy_url = "https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=" + str(i) +"&per-page=50"
    print(policy_url)
    print(i)

https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=1&per-page=50
1
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=2&per-page=50
2
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=3&per-page=50
3
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=4&per-page=50
4
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=5&per-page=50
5
https://policy.bangkok.go.th/tracking/frontend/web/index.php?r=site%2Findexcheckreportkpi2&page=6&per-page=50
6


In [ ]:
progress_table[0]

,#,KEY_RESULT,ค่าเป้าหมาย/ปี,ผลดำเนินงาน (รวม),หน่วยนับ,ตค.66,พย.66,ธค.66,มค.67,กพ.67,มี.ค.67,เม.ย.67,พค.67,มิ.ย.67,กค.67,สค.67,กย.67
0,1.0,1. พัฒนาถนนสวย (กม.) : OKR 4.1.10 99.00%,123.6,122.36,กม.,2.27,0.2,3.49,17.01,20.46,26.27,17.6,12.39,7.65,15.03,0.0,0.0
1,2.0,2. ปลูกต้นไม้รวมทุกประเภท (ต้น) : OKR 4.1.01 145.43%,200000.0,290856.0,ต้น,8734.0,24454.0,15086.0,29427.0,38176.0,51876.0,29470.0,25710.0,35950.0,31973.0,0.0,0.0
2,3.0,3. เพิ่มสวน 15 นาที (แห่ง) : OKR 4.1.05 77.24%,156.0,120.5,แห่ง,2.0,6.0,7.0,16.0,13.0,22.0,11.0,9.0,14.0,20.5,0.0,0.0
3,4.0,4.1 ปรับปรุงทางเท้า (กม.) : OKR 1.1.05 36.32%,312.07,113.35,กม.,0.0,0.0,0.0,1.4,0.75,2.98,5.77,5.31,3.63,93.51,0.0,0.0
4,5.0,5. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่าง (ดวง) : OKR 2.1.20 123.20%,31878.0,39273.0,ดวง,2103.0,3578.0,4445.0,4341.0,3876.0,6195.0,3985.0,4336.0,3828.0,2586.0,0.0,0.0
5,6.0,6. ติดตั้ง/ซ่อม ไฟฟ้าส่องสว่างริมคลอง (ดวง) : OKR 1.1.03 48.62%,9536.0,4636.28,ดวง,70.0,316.0,234.0,325.0,1413.0,345.0,507.0,472.0,416.0,538.28,0.0,0.0
6,7.0,7. ยกเลิก/ยุบรวม พื้นที่หาบเร่แผงลอยนอกจุดผ่อนผัน (แห่ง) : OKR 1.1.01 116.00%,100.0,116.0,แห่ง,3.0,1.0,1.0,14.0,14.0,30.0,23.0,12.0,7.0,11.0,0.0,0.0
7,8.0,8. จัดหาพื้นที่เอกชน หรือ พัฒนา Hawker Center รองรับผู้ค้าหาบเร่ (แห่ง) : OKR 7.2.05 272.73%,11.0,30.0,แห่ง,1.0,3.0,1.0,4.0,2.0,7.0,5.0,4.0,2.0,1.0,0.0,0.0
8,9.0,9. แก้ไขจุดเสี่ยงน้ำท่วม (จุด) : OKR 1.4.05 103.96%,212.0,220.4,จุด,5.0,22.0,1.0,5.0,8.01,32.0,12.2,17.2,7.45,110.54,0.0,0.0
9,10.0,10.1 แก้ปัญหาจุดจราจรฝืด (แห่ง) : OKR 1.2.05 63.77%,265.0,169.0,แห่ง,7.0,3.0,6.0,11.0,12.0,38.0,39.0,28.0,18.0,7.0,0.0,0.0


## All code of joining the dataframes

In [ ]:
df_rushing_policy_for_join = pd.read_parquet('df_rushing_policies.parquet', columns=['ID_Result','Goal'])
df_progress_for_join = pd.read_parquet('all_progress.parquet', columns = ['ID_Result', 'Yearly_Goal', 'Total_Progress_Unit', 'Unit', 'Total_Progress_in_Percent', 'Oct 23',
       'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24', 'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24'])
df_joined = df_rushing_policy_for_join.merge(df_progress_for_join, left_on = 'ID_Result', right_on = 'ID_Result', how = 'left')

today = datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%d-%m-%Y")
df_joined['Updated_Date'] = today
df_joined = df_joined[['Updated_Date','ID_Result', 'Goal', 'Yearly_Goal', 'Total_Progress_Unit', 'Unit',
       'Total_Progress_in_Percent', 'Oct 23', 'Nov 23', 'Dec 23', 'Jan 24', 'Feb 24',
       'Mar 24', 'Apr 24', 'May 24', 'Jun 24', 'July 24', 'Aug 24', 'Sept 24',]]

In [ ]:
all_months = ['Oct_23', 'Nov_23', 'Dec_23', 'Jan_24', 'Feb_24', 'Mar_24', 'Apr_24', 'May_24', 'Jun_24', 'July_24', 'Aug_24', 'Sept_24']

for month in all_months:
    df_progress[month + '_Percent'] = df_progress[month].div(df_progress.Yearly_Goal, axis=0)*100
    df_progress[month + '_Percent'] = df_progress[month + '_Percent'].round(2)

In [ ]:
df_joined.head()

,Updated_Date,No. (Goal),Goal,Yearly Goal,Total Progress (Unit),Unit,Total Progress (%),Oct 23,Nov 23,Dec 23,Jan 24,Feb 24,Mar 24,Apr 24,May 24,Jun 24,July 24,Aug 24,Sept 24
0,10-08-2024,1,พัฒนาถนนสวย (กม.),123.60,129.05,กม.,104.41%,2.27,0.2,3.49,17.01,20.46,26.27,17.60,12.39,9.15,20.22,0.00,0.0
1,10-08-2024,2,ปลูกต้นไม้รวมทุกประเภท (ต้น),200000.00,362942.00,ต้น,181.47%,8821.00,24435.0,15020.00,29573.00,38701.00,51849.00,29556.00,25902.00,36020.00,95836.00,7229.00,0.0
2,10-08-2024,3,เพิ่มสวน 15 นาที (แห่ง),156.00,130.00,แห่ง,83.33%,2.00,6.0,7.00,16.00,13.00,22.00,11.00,9.00,20.00,20.00,4.00,0.0
3,10-08-2024,4.1,ปรับปรุงทางเท้า (กม.),312.07,115.47,กม.,37.00%,0.00,0.0,0.00,1.40,0.75,2.98,5.77,5.31,4.53,94.49,0.24,0.0
4,10-08-2024,4.2,ทางจักรยาน (กม.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Traffy Fondue REST API Request

In [ ]:
import pandas as pd

In [ ]:
traffy_url = 'https://publicapi.traffy.in.th/teamchadchart-stat-api/geojson/v1?output_format=csv&name=Sukatat&org=Non&purpose=educational_project&email=skt.sukatat@gmail.com&limit=&sort=DESC'

df_traffy = pd.read_csv(traffy_url, columns = ['ticket_id', 'type','organization'])
df_traffy.head()

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,2024-P7WZWP,อื่นๆ,"กรุงเทพมหานคร, เขตบางขุนเทียน","เขตบางขุนเทียน, กรุงเทพมหานคร",มีรถจอดแช่ทิ้งเป็นเศษเหล็กไม่ขยับ และกันทางเข้...,"100.44833,13.65658",https://storage.googleapis.com/traffy_public_b...,NaN,1 ซ. อนามัยงามเจริญ 19 แขวงท่าข้าม เขตบางขุนเท...,ท่าข้าม,...,2024-08-05 22:18:39,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-6Z76BP,อื่นๆ,"กรุงเทพมหานคร, เขตพญาไท","เขตพญาไท, กรุงเทพมหานคร",ไฟส่องสว่างเสีย ป้ายรถเมล ซอยราชครู,"100.54338,13.77654",https://storage.googleapis.com/traffy_public_b...,NaN,1024 ถ. พหลโยธิน แขวงพญาไท เขตพญาไท กรุงเทพมหา...,สามเสนใน,...,2024-08-05 22:15:07,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-8UFFNF,อื่นๆ,"กรุงเทพมหานคร, เขตสวนหลวง","เขตสวนหลวง, กรุงเทพมหานคร",แจ้งปัญหาเรื่องเสียงดังจากวัดยาง อ่อนนุช 17\n\...,"100.62024,13.71221",https://storage.googleapis.com/traffy_public_b...,NaN,44 ถ. อ่อนนุช แขวงสวนหลวง เขตสวนหลวง กรุงเทพมห...,สวนหลวง,...,2024-08-05 22:13:52,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-4KGDKF,อื่นๆ,"กรุงเทพมหานคร, เขตปทุมวัน","เขตปทุมวัน, กรุงเทพมหานคร",รถสามล้อรถแท๊กซี่เข้ามารอรับผู้โดยสารต่างชาติบ...,"100.54051,13.74679",https://storage.googleapis.com/traffy_public_b...,NaN,บิ๊กซีราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุงเทพม...,ปทุมวัน,...,2024-08-05 22:09:34,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-7MW4W2,อื่นๆ,"กรุงเทพมหานคร, เขตสวนหลวง","เขตสวนหลวง, กรุงเทพมหานคร",ได้รับตอบรับว่าการแก้ไขปัญหาที่เคยร้องเรียนไป ...,"100.63779,13.70678",https://storage.googleapis.com/traffy_public_b...,NaN,419 ซอย ผ่องโสภา แขวงสวนหลวง เขตสวนหลวง กรุงเท...,สวนหลวง,...,2024-08-05 22:08:32,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [ ]:
df_traffy[:1000]

,ticket_id,type,organization,organization_action,comment,coords,photo,photo_after,address,subdistrict,...,last_activity,duration_minutes_inprogress,duration_minutes_finished,duration_minutes_total,timestamp_inprogress,timestamp_finished,view_count,total_point,like,dislike
0,2024-P7WZWP,อื่นๆ,"กรุงเทพมหานคร, เขตบางขุนเทียน","เขตบางขุนเทียน, กรุงเทพมหานคร",มีรถจอดแช่ทิ้งเป็นเศษเหล็กไม่ขยับ และกันทางเข้...,"100.44833,13.65658",https://storage.googleapis.com/traffy_public_b...,NaN,1 ซ. อนามัยงามเจริญ 19 แขวงท่าข้าม เขตบางขุนเท...,ท่าข้าม,...,2024-08-05 22:18:39,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,2024-6Z76BP,อื่นๆ,"กรุงเทพมหานคร, เขตพญาไท","เขตพญาไท, กรุงเทพมหานคร",ไฟส่องสว่างเสีย ป้ายรถเมล ซอยราชครู,"100.54338,13.77654",https://storage.googleapis.com/traffy_public_b...,NaN,1024 ถ. พหลโยธิน แขวงพญาไท เขตพญาไท กรุงเทพมหา...,สามเสนใน,...,2024-08-05 22:15:07,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,2024-8UFFNF,อื่นๆ,"กรุงเทพมหานคร, เขตสวนหลวง","เขตสวนหลวง, กรุงเทพมหานคร",แจ้งปัญหาเรื่องเสียงดังจากวัดยาง อ่อนนุช 17\n\...,"100.62024,13.71221",https://storage.googleapis.com/traffy_public_b...,NaN,44 ถ. อ่อนนุช แขวงสวนหลวง เขตสวนหลวง กรุงเทพมห...,สวนหลวง,...,2024-08-05 22:13:52,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,2024-4KGDKF,อื่นๆ,"กรุงเทพมหานคร, เขตปทุมวัน","เขตปทุมวัน, กรุงเทพมหานคร",รถสามล้อรถแท๊กซี่เข้ามารอรับผู้โดยสารต่างชาติบ...,"100.54051,13.74679",https://storage.googleapis.com/traffy_public_b...,NaN,บิ๊กซีราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุงเทพม...,ปทุมวัน,...,2024-08-05 22:09:34,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,2024-7MW4W2,อื่นๆ,"กรุงเทพมหานคร, เขตสวนหลวง","เขตสวนหลวง, กรุงเทพมหานคร",ได้รับตอบรับว่าการแก้ไขปัญหาที่เคยร้องเรียนไป ...,"100.63779,13.70678",https://storage.googleapis.com/traffy_public_b...,NaN,419 ซอย ผ่องโสภา แขวงสวนหลวง เขตสวนหลวง กรุงเท...,สวนหลวง,...,2024-08-05 22:08:32,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-CF6EVC,อื่นๆ,"กรุงเทพมหานคร, เขตหลักสี่, ฝ่ายโยธา เขตหลักสี่","ฝ่ายโยธา เขตหลักสี่, เขตหลักสี่, กรุงเทพมหานคร",มีความเย็น จากการใช้แอร์ \nในส่วนที่ต่อเติม ด้...,"100.54808,13.85436",https://storage.googleapis.com/traffy_public_b...,NaN,ซอย งามวงศ์วาน 30 ถนน งามวงศ์วาน แขวง ทุ่งสองห...,ทุ่งสองห้อง,...,2024-08-05 07:46:10,644.0,NaN,644.0,2024-08-05 07:46:38,NaN,0,0,0,0
996,2024-KANRK8,อื่นๆ,"กรุงเทพมหานคร, เขตประเวศ, ฝ่ายโยธา เขตประเวศ","ฝ่ายโยธา เขตประเวศ, เขตประเวศ, กรุงเทพมหานคร",** แจ้งไปหลายครั้งแล้ว ยังไม่ได้รับการแก้ไขเลย...,"100.68957,13.66513",https://storage.googleapis.com/traffy_public_b...,NaN,7 รามคำแหง 2 ซอย 6 แขวง ดอกไม้ เขต ประเวศ กรุง...,ดอกไม้,...,2024-08-05 07:58:11,200.0,NaN,200.0,2024-08-05 00:19:41,NaN,0,0,0,0
997,2024-DGZ7TL,อื่นๆ,"กรุงเทพมหานคร, เขตบางพลัด, สน.บางพลัด, กองบัญช...","เขตบางพลัด, กองบังคับการตำรวจนครบาล 7 (บก.น.7)...",กลางคืน มีรถจอดกีดขวางทางสัญจรในซอยแคบและใกล้ท...,"100.49499,13.79557",https://storage.googleapis.com/traffy_public_b...,NaN,1160 จรัญสนิทวงศ์ 75 แยก 6 แขวงบางพลัด เขตบางพ...,บางพลัด,...,2024-08-04 20:59:14,NaN,NaN,NaN,NaN,NaN,0,0,0,0
998,2024-GUGRUA,อื่นๆ,"กรุงเทพมหานคร, เขตประเวศ, ฝ่ายโยธา เขตประเวศ","เขตประเวศ, ฝ่ายโยธา เขตประเวศ, กรุงเทพมหานคร",เจ้าหน้าที่เขต ช่วยกำชับพนักงานที่ทำหน้าที่ควบ...,"100.69078,13.6699",https://storage.googleapis.com/traffy_public_b...,NaN,MM9R+X89 ซอย รามคำแหง 2 แขวงดอกไม้ เขต ประเวศ ...,ดอกไม้,...,2024-08-05 00:19:59,206.0,NaN,206.0,2024-08-05 00:20:42,NaN,0,0,0,0


In [ ]:
df_traffy.columns

Index(['ticket_id', 'type', 'organization', 'organization_action', 'comment',
       'coords', 'photo', 'photo_after', 'address', 'subdistrict', 'district',
       'province', 'timestamp', 'state', 'star', 'count_reopen',
       'last_activity', 'duration_minutes_inprogress',
       'duration_minutes_finished', 'duration_minutes_total',
       'timestamp_inprogress', 'timestamp_finished', 'view_count',
       'total_point', 'like', 'dislike'],
      dtype='object')

In [ ]:
df_traffy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticket_id                    1000 non-null   object 
 1   type                         1000 non-null   object 
 2   organization                 1000 non-null   object 
 3   organization_action          1000 non-null   object 
 4   comment                      1000 non-null   object 
 5   coords                       1000 non-null   object 
 6   photo                        1000 non-null   object 
 7   photo_after                  85 non-null     object 
 8   address                      1000 non-null   object 
 9   subdistrict                  1000 non-null   object 
 10  district                     1000 non-null   object 
 11  province                     1000 non-null   object 
 12  timestamp                    1000 non-null   object 
 13  state              

In [ ]:
df_traffy.value_counts('type')

,count
type,
อื่นๆ,660
ไฟฟ้า,42
ทางเท้า,39
ความสะอาด,31
ถนน,31
น้ำท่วม,23
ผิดกฎจราจร,23
เสียง,20
ต้นไม้,14
